<a href="https://colab.research.google.com/github/Dermapp/DataPreprocessing/blob/master/Inceptionv3model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id' : '1egI02bfGIzP47tDnHnGU2_W1dXRi0Axc'})
download.GetContentFile('skin_image.zip')
! unzip skin_image.zip 

In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
# create the base pre-trained model
input_tensor = Input(shape=(224, 224, 3)) 
base_model = InceptionV3(input_tensor=input_tensor,weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(128, activation='relu')(x)
# and a logistic layer
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

87916544/87910968 [==============================] - 7s 0us/step


In [4]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 111, 111, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 111, 111, 32) 0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

In [0]:
import cv2
img = cv2.imread('/content/skin_image/train/akeic/ISIC_0024372.jpg')

In [15]:
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('skin_image/train',
                                                 target_size = (224,224),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('skin_image/validation',
                                            target_size = (224,224),
                                            batch_size = 8,
                                            class_mode = 'categorical')
labels = (training_set.class_indices)


Found 7019 images belonging to 7 classes.
Found 1508 images belonging to 7 classes.


In [16]:
print(labels)

{'akeic': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}


In [17]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [19]:
history = model.fit_generator(training_set,
                         steps_per_epoch = 7019,
                         epochs = 1,
                         validation_data = test_set,
                         validation_steps = 1708)

Epoch 1/1
  12/7019 [..............................] - ETA: 12:34:43 - loss: 1.6622 - acc: 0.5000

KeyboardInterrupt: ignored